In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
! pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import geocoder # import geocoder
import math
import re
print('Libraries imported.')

Libraries imported.


In [2]:
CLIENT_ID = 'YA1V0ZT4RLWE3PVLWULDI0QLPHCUNN0BNXSOT5GS3P1EZR0G' # your Foursquare ID
CLIENT_SECRET = 'ITTIADOFH3REMVNUNONWLMRG0T2051ETQD5GCDGDXBWNRJY5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YA1V0ZT4RLWE3PVLWULDI0QLPHCUNN0BNXSOT5GS3P1EZR0G
CLIENT_SECRET:ITTIADOFH3REMVNUNONWLMRG0T2051ETQD5GCDGDXBWNRJY5


In [3]:
path = "/Users/isalau/Documents/GitHub/Coursera_Capstone/Neighborhood_Labels.csv"
dc = pd.read_csv(path)  

In [4]:
dc = dc.rename({'X': 'Longitude'}, axis='columns')
dc = dc.rename({'Y': 'Latitude'}, axis='columns')
dc = dc.rename({'NAME': 'Neighborhood'}, axis='columns')

In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [6]:
names=dc['Neighborhood']
latitudes=dc['Latitude']
longitudes=dc['Longitude']
radius=500
LIMIT=100
for name, lat, lng in zip(names, latitudes, longitudes):            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]

In [7]:
path = "/Users/isalau/Documents/GitHub/Coursera_Capstone/dc_venues.csv"
dc_venues_from_csv= pd.read_csv(path)
dc_venues_from_csv.head()
dc_venues = dc_venues_from_csv

In [8]:
# dc_venues = getNearbyVenues(names=dc['Neighborhood'],
#                                    latitudes=dc['Latitude'],
#                                    longitudes=dc['Longitude']
#                                   )
# dc_venues.to_csv('dc_venues.csv', index=False, header=True)

In [9]:
dc_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(dc_venues['Venue Category'].unique())))

There are 308 uniques categories.


In [10]:
# one hot encoding
dc_onehot = pd.get_dummies(dc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dc_onehot['Neighborhood'] = dc_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dc_onehot.columns[-1]] + list(dc_onehot.columns[:-1])
dc_onehot = dc_onehot[fixed_columns]

In [11]:
dc_grouped = dc_onehot.groupby('Neighborhood').mean().reset_index()

In [12]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [13]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dc_grouped['Neighborhood']

for ind in np.arange(dc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dc_grouped.iloc[ind, :], num_top_venues)

# Manully Cluster Neighborhoods

In [14]:
categories = dc_venues
cat = categories.drop(['Neighborhood Latitude','Neighborhood Longitude', 'Venue Latitude','Venue Longitude'], axis=1)
print('There are originally {} uniques categories.'.format(len(cat['Venue Category'].unique())))


There are originally 308 uniques categories.


General venue categories relabeled

In [15]:
#Residential
Grocery = ['Tea','Bakery','Bagel Shop','Candy Store','Chocolate Shop','Convenience Store','Cupcake Shop','Deli / Bodega','Dessert Shop','Donut Shop','Drugstore','Farmers Market','Food,Food & Drink Shop','Gourmet Shop','Grocery Store','Liquor Store','Market','Organic Grocery','Pharmacy','Smoothie Shop','Snack Place','Supermarket','Supplement Shop','Wine Shop','Beer Store']
Health = ['Chiropractor','Health & Beauty Service','Hospital','Massage Studio','Nail Salon','Spa','Salon / Barbershop']
Home = ['Dog Run','Dry Cleaner','Home Service','Laundromat','Locksmith','Moving Target','Other Repair Shop','Pet Service','Shoe Repair','Storage Facility','Tailor Shop','Residential Building (Apartment / Condo)']
Sports = ['Athletics & Sports','Baseball Field','Basketball Court','Basketball Stadium','Boxing Gym','Cycle Studio','Dance Studio','Field','Golf Course','Gym','Gym / Fitness Center','Gym Pool','Gymnastics Gym','Harbor / Marina','Heliport','Lake','Martial Arts School','Pilates Studio','Playground','Skating Rink','Soccer Field','Soccer Stadium','Sports Club','State / Provincial Park','Volleyball Court','Yoga Studio','Recreation Center','Track','Tennis Court']

#Residential/Tourism 
Shopping=['Accessories Store','Antique Shop','Arts & Crafts Store','Automotive Shop','Big Box Store','Bookstore','Boutique','Camera Store','Christmas Market','Clothing Store','Comic Shop','Cosmetics Shop','Department Store','Discount Store','Electronics Store','Flea Market','Flower Shop','Furniture / Home Store','Gift Shop','Hardware Store','Herbs & Spices Store','Jewelry Store','Kids Store','Kitchen Supply Store','Leather Goods Store','Lingerie Store','Mattress Store',"Men/'s Store",'Miscellaneous Shop','Mobile Phone Shop','Music Store','Optical Shop','Paper / Office Supplies Store','Pet Store','Photography Lab','Record Shop','Shipping Store','Shoe Store','Shop & Service','Shopping Mall','Shopping Plaza','Smoke Shop','Souvenir Shop','Sporting Goods Shop','Thrift / Vintage Store','Toy / Game Store','Video Store','Warehouse Store',"Women/'s Store"]
Bar = ['Bar','Beer Bar','Beer Garden','Brewery','Cocktail Bar','Dive Bar','Gastropub','Gay Bar','Irish Pub','Karaoke Bar','Pub','Speakeasy','Sports Bar','Thai Restaurant','Tiki Bar','Whisky Bar','Wine Bar']
Restaurant = ['Afghan Restaurant','American Restaurant','Arepa Restaurant','Asian Restaurant','BBQ Joint','Belgian Restaurant','Bistro','Brazilian Restaurant','Breakfast Spot','Burger Joint','Burrito Place','Café','Cafeteria','Cajun / Creole Restaurant','Cantonese Restaurant','Caribbean Restaurant','Cheese Shop','Chinese Restaurant','Coffee Shop','Comfort Food Restaurant','Cuban Restaurant','Diner','Dumpling Restaurant','Eastern European Restaurant','Empanada Restaurant','Ethiopian Restaurant','Falafel Restaurant','Fast Food Restaurant','Filipino Restaurant','Fish & Chips Shop','Food Court','Food Service','Food Truck','French Restaurant','Fried Chicken Joint','Frozen Yogurt Shop','German Restaurant','Gluten-free Restaurant','Greek Restaurant','Hot Dog Joint','Ice Cream Shop','Indian Restaurant','Israeli Restaurant','Italian Restaurant','Japanese Restaurant','Juice Bar','Korean Restaurant','Latin American Restaurant','Mediterranean Restaurant','Mexican Restaurant','Middle Eastern Restaurant','New American Restaurant','Noodle House','Peruvian Restaurant','Pizza Place','Poke Place','Portuguese Restaurant','Puerto Rican Restaurant','Ramen Restaurant','Restaurant','Salad Place','Salvadoran Restaurant','Sandwich Place','Scandinavian Restaurant','Seafood Restaurant','South American Restaurant','Southern / Soul Food Restaurant','Spanish Restaurant','Steakhouse','Street Food Gathering','Sushi Restaurant','Taco Place','Tapas Restaurant','Tea Room','Tex-Mex Restaurant','Turkish Restaurant','Vegetarian / Vegan Restaurant','Vietnamese Restaurant','Wings Joint','Xinjiang Restaurant']
Transportation = ['Bike Rental / Bike Share','Boat or Ferry','Border Crossing','Bridge','Bus Line','Bus Station','Bus Stop','Gas Station','Metro Station','Rental Car Location','Trail','Train Station','Tunnel']
Entertainment = ['Bowling','Hookah Bar','Lounge','Nightclub','Nightlife Spot','Roof Deck','Event Space']
Banking = ['ATM','Bank','Check Cashing Service','Credit Union','Banking']
Culture = ['Art Gallery','Art Museum','Botanical Garden','Comedy Club','Escape Room','Exhibit','Fountain','Garden','General Entertainment','Historic Site','History Museum','Indie Movie Theater','Memorial Site','Monument / Landmark','Movie Theater','Museum','Music Venue','Opera House','Outdoor Sculpture','Park','Pedestrian Plaza','Performing Arts Venue','Planetarium','Plaza','Public Art','Rock Club','Scenic Lookout','Science Museum','Sculpture Garden','Synagogue','Theater','Winery','Zoo Exhibit']

#Tourism
Lodging = ['Bed & Breakfast','Hostel','Hotel','Hotel Bar','Hotel Pool']

#Industrial
Infrastructure = ['Intersection','Reservoir','River','Road']
Business = ['Building','Business Service','Construction & Landscaping','Government Building','Insurance Office','Lawyer','Light Rail Station','Office','Post Office']

In [16]:
cat = cat.replace(to_replace= Grocery, value="Grocery", regex = True)
cat = cat.replace(to_replace= Health, value="Health", regex = True)
cat = cat.replace(to_replace= Home, value="Home", regex = True)
cat = cat.replace(to_replace= Sports, value="Sports", regex = True)

cat = cat.replace(to_replace= Shopping, value="Shopping", regex = True)
cat = cat.replace(to_replace= Restaurant, value="Restaurant", regex = True)
cat = cat.replace(to_replace= Bar, value="Bar", regex = True)
cat = cat.replace(to_replace= Transportation, value="Transportation", regex = True)
cat = cat.replace(to_replace= Entertainment, value="Entertainment", regex = True)
cat = cat.replace(to_replace= Banking, value="Banking", regex = True)
cat = cat.replace(to_replace= Culture, value="Culture", regex = True)


cat = cat.replace(to_replace= Lodging, value="Lodging", regex = True)

cat = cat.replace(to_replace= Infrastructure, value="Infrastructure", regex = True)
cat = cat.replace(to_replace= Business, value="Business", regex = True)

print('There are {} unique categories after dictionary switch.'.format(len(cat['Venue Category'].unique())))
print('There are {} unique categories after dictionary switch.'.format((cat['Venue Category'].unique())))

There are 61 unique categories after dictionary switch.
There are ['Culture' 'Sports' 'Infrastructure' 'Grocery' 'Restaurant' 'Food' 'Home'
 'Banking' 'Shopping' 'Health' "Women's Store" 'Lodging' 'Bar' 'Pool'
 'Sports / Fitness Center' 'Flea Grocery' 'New Restaurant'
 'Latin Restaurant' 'Healthnish Restaurant' 'Transportation'
 'College Administrative Business' 'South Restaurant' 'Soccer Sports'
 'Neighborhood' 'Business' 'Concert Hall' 'Entertainment'
 'Child Care Service' 'Sculpture Culture' 'College Sports' "Men's Store"
 'Grocery Room' 'Christmas Grocery' 'Tour Provider'
 'General College & University' 'Barlic Art' 'Bankinging'
 'Residential Business (Apartment / Condo)' 'Sports Pool'
 'College Shopping' 'Stables' 'Food & Drink Shop' 'Sportsnastics Sports'
 'Government Business' 'Event Healthce' 'Lodging Bar' 'College Quad'
 'Entertainment Green' 'Pet Restaurant' 'Science Culture' 'Lodging Pool'
 'Photography Studio' 'Frame Store' 'Tourist Information Center'
 'Zoo Culture' 'Bubbl

In [17]:
#anything missed shuld be caught here

#Residential
cat = cat.replace(regex=r'.*Food.*', value="Grocery")
cat = cat.replace(regex=r'.*Grocery.*', value="Grocery")


cat = cat.replace(regex=r'.*Auto.*', value="Home")
cat = cat.replace(regex=r'.*School.*', value="Home")
cat = cat.replace(regex=r'.*Residential.*', value="Home")
cat = cat.replace(regex=r'.*Child.*', value="Home")
cat = cat.replace(regex=r'.*Care.*', value="Home")
cat = cat.replace(regex=r'.*Home.*', value="Home")
cat = cat.replace(regex=r'.*Neighborhood.*', value="Home")

cat = cat.replace(regex=r'.*Sports.*', value="Sports")
cat = cat.replace(regex=r'.*Studio.*', value="Sports")
cat = cat.replace(regex=r'.*Stables.*', value="Sports")
cat = cat.replace(regex=r'.*Store.*', value="Sports")

#Residential/Tourism
cat = cat.replace(regex=r'.*Bar.*', value="Bar")
cat = cat.replace(regex=r'.*Restaurant.*', value="Restaurant")

cat = cat.replace(regex=r'.*Men.*', value="Shopping")
cat = cat.replace(regex=r'.*Women.*', value="Shopping")

cat = cat.replace(regex=r'.*Culture.*', value="Culture")
cat = cat.replace(regex=r'.*Event.*', value="Culture")
cat = cat.replace(regex=r'.*Concert.*', value="Culture")
cat = cat.replace(regex=r'.*Art.*', value="Culture")

cat = cat.replace(regex=r'.*Pool.*', value="Entertainment")
cat = cat.replace(regex=r'.*Entertainment.*', value="Entertainment")


#Tourism 
cat = cat.replace(regex=r'.*Tour.*', value="Tourism")
cat = cat.replace(regex=r'.*Hotel.*', value="Lodging")

#Industrial 
cat = cat.replace(regex=r'.*College.*', value="College")
cat = cat.replace(regex=r'.*Business.*', value="Business")
cat = cat.replace(regex=r'.*Bank.*', value="Banking")


In [18]:
print('There are {} final unique categories.'.format(len(cat['Venue Category'].unique())))
print('There are {} final unique categories.'.format((cat['Venue Category'].unique())))

There are 17 final unique categories.
There are ['Culture' 'Sports' 'Infrastructure' 'Grocery' 'Restaurant' 'Home'
 'Banking' 'Shopping' 'Health' 'Lodging' 'Bar' 'Entertainment'
 'Transportation' 'College' 'Business' 'Tourism' 'Factory'] final unique categories.


In [19]:
l = cat['Venue Category'].unique()
l

array(['Culture', 'Sports', 'Infrastructure', 'Grocery', 'Restaurant',
       'Home', 'Banking', 'Shopping', 'Health', 'Lodging', 'Bar',
       'Entertainment', 'Transportation', 'College', 'Business',
       'Tourism', 'Factory'], dtype=object)

For every neighborhood assign it either Residential, Tourism or Industrial based on majority type of venue

In [20]:
labels = cat

In [21]:
print('There are {} final unique categories.'.format(len(labels['Venue Category'].unique())))
print('There are {} final unique categories.'.format((labels['Venue Category'].unique())))

There are 17 final unique categories.
There are ['Culture' 'Sports' 'Infrastructure' 'Grocery' 'Restaurant' 'Home'
 'Banking' 'Shopping' 'Health' 'Lodging' 'Bar' 'Entertainment'
 'Transportation' 'College' 'Business' 'Tourism' 'Factory'] final unique categories.


In [22]:
labels = labels.replace(regex=r'Grocery|Health|Home|Sports', value="Residential")
labels = labels.replace(regex=r'Shopping|Bar|Restaurant|Culture|Banking|Entertainment|Transportation', value="RTBoth")
labels = labels.replace(regex=r'Lodging|Tourism', value="Tourism")
labels = labels.replace(regex=r'Business|Infrastructure|College|Factory', value="Industrial")

print('There are {} final unique categories.'.format(len(labels['Venue Category'].unique())))
print('There are {} final unique categories.'.format((labels['Venue Category'].unique())))

There are 4 final unique categories.
There are ['RTBoth' 'Residential' 'Industrial' 'Tourism'] final unique categories.


In [31]:
def rename_labels(cat):
    #Residential
    Grocery = ['Tea','Bakery','Bagel Shop','Candy Store','Chocolate Shop','Convenience Store','Cupcake Shop','Deli / Bodega','Dessert Shop','Donut Shop','Drugstore','Farmers Market','Food,Food & Drink Shop','Gourmet Shop','Grocery Store','Liquor Store','Market','Organic Grocery','Pharmacy','Smoothie Shop','Snack Place','Supermarket','Supplement Shop','Wine Shop','Beer Store']
    Health = ['Chiropractor','Health & Beauty Service','Hospital','Massage Studio','Nail Salon','Spa','Salon / Barbershop']
    Home = ['Dog Run','Dry Cleaner','Home Service','Laundromat','Locksmith','Moving Target','Other Repair Shop','Pet Service','Shoe Repair','Storage Facility','Tailor Shop','Residential Building (Apartment / Condo)']
    Sports = ['Athletics & Sports','Baseball Field','Basketball Court','Basketball Stadium','Boxing Gym','Cycle Studio','Dance Studio','Field','Golf Course','Gym','Gym / Fitness Center','Gym Pool','Gymnastics Gym','Harbor / Marina','Heliport','Lake','Martial Arts School','Pilates Studio','Playground','Skating Rink','Soccer Field','Soccer Stadium','Sports Club','State / Provincial Park','Volleyball Court','Yoga Studio','Recreation Center','Track','Tennis Court']

    #Residential/Tourism 
    Shopping=['Accessories Store','Antique Shop','Arts & Crafts Store','Automotive Shop','Big Box Store','Bookstore','Boutique','Camera Store','Christmas Market','Clothing Store','Comic Shop','Cosmetics Shop','Department Store','Discount Store','Electronics Store','Flea Market','Flower Shop','Furniture / Home Store','Gift Shop','Hardware Store','Herbs & Spices Store','Jewelry Store','Kids Store','Kitchen Supply Store','Leather Goods Store','Lingerie Store','Mattress Store',"Men/'s Store",'Miscellaneous Shop','Mobile Phone Shop','Music Store','Optical Shop','Paper / Office Supplies Store','Pet Store','Photography Lab','Record Shop','Shipping Store','Shoe Store','Shop & Service','Shopping Mall','Shopping Plaza','Smoke Shop','Souvenir Shop','Sporting Goods Shop','Thrift / Vintage Store','Toy / Game Store','Video Store','Warehouse Store',"Women/'s Store"]
    Bar = ['Bar','Beer Bar','Beer Garden','Brewery','Cocktail Bar','Dive Bar','Gastropub','Gay Bar','Irish Pub','Karaoke Bar','Pub','Speakeasy','Sports Bar','Thai Restaurant','Tiki Bar','Whisky Bar','Wine Bar']
    Restaurant = ['Afghan Restaurant','American Restaurant','Arepa Restaurant','Asian Restaurant','BBQ Joint','Belgian Restaurant','Bistro','Brazilian Restaurant','Breakfast Spot','Burger Joint','Burrito Place','Café','Cafeteria','Cajun / Creole Restaurant','Cantonese Restaurant','Caribbean Restaurant','Cheese Shop','Chinese Restaurant','Coffee Shop','Comfort Food Restaurant','Cuban Restaurant','Diner','Dumpling Restaurant','Eastern European Restaurant','Empanada Restaurant','Ethiopian Restaurant','Falafel Restaurant','Fast Food Restaurant','Filipino Restaurant','Fish & Chips Shop','Food Court','Food Service','Food Truck','French Restaurant','Fried Chicken Joint','Frozen Yogurt Shop','German Restaurant','Gluten-free Restaurant','Greek Restaurant','Hot Dog Joint','Ice Cream Shop','Indian Restaurant','Israeli Restaurant','Italian Restaurant','Japanese Restaurant','Juice Bar','Korean Restaurant','Latin American Restaurant','Mediterranean Restaurant','Mexican Restaurant','Middle Eastern Restaurant','New American Restaurant','Noodle House','Peruvian Restaurant','Pizza Place','Poke Place','Portuguese Restaurant','Puerto Rican Restaurant','Ramen Restaurant','Restaurant','Salad Place','Salvadoran Restaurant','Sandwich Place','Scandinavian Restaurant','Seafood Restaurant','South American Restaurant','Southern / Soul Food Restaurant','Spanish Restaurant','Steakhouse','Street Food Gathering','Sushi Restaurant','Taco Place','Tapas Restaurant','Tea Room','Tex-Mex Restaurant','Turkish Restaurant','Vegetarian / Vegan Restaurant','Vietnamese Restaurant','Wings Joint','Xinjiang Restaurant']
    Transportation = ['Bike Rental / Bike Share','Boat or Ferry','Border Crossing','Bridge','Bus Line','Bus Station','Bus Stop','Gas Station','Metro Station','Rental Car Location','Trail','Train Station','Tunnel']
    Entertainment = ['Bowling','Hookah Bar','Lounge','Nightclub','Nightlife Spot','Roof Deck','Event Space']
    Banking = ['ATM','Bank','Check Cashing Service','Credit Union','Banking']
    Culture = ['Art Gallery','Art Museum','Botanical Garden','Comedy Club','Escape Room','Exhibit','Fountain','Garden','General Entertainment','Historic Site','History Museum','Indie Movie Theater','Memorial Site','Monument / Landmark','Movie Theater','Museum','Music Venue','Opera House','Outdoor Sculpture','Park','Pedestrian Plaza','Performing Arts Venue','Planetarium','Plaza','Public Art','Rock Club','Scenic Lookout','Science Museum','Sculpture Garden','Synagogue','Theater','Winery','Zoo Exhibit']

    #Tourism
    Lodging = ['Bed & Breakfast','Hostel','Hotel','Hotel Bar','Hotel Pool']

    #Industrial
    Infrastructure = ['Intersection','Reservoir','River','Road']
    Business = ['Building','Business Service','Construction & Landscaping','Government Building','Insurance Office','Lawyer','Light Rail Station','Office','Post Office']

    cat = cat.replace(to_replace= Grocery, value="Grocery", regex = True)
    cat = cat.replace(to_replace= Health, value="Health", regex = True)
    cat = cat.replace(to_replace= Home, value="Home", regex = True)
    cat = cat.replace(to_replace= Sports, value="Sports", regex = True)

    cat = cat.replace(to_replace= Shopping, value="Shopping", regex = True)
    cat = cat.replace(to_replace= Restaurant, value="Restaurant", regex = True)
    cat = cat.replace(to_replace= Bar, value="Bar", regex = True)
    cat = cat.replace(to_replace= Transportation, value="Transportation", regex = True)
    cat = cat.replace(to_replace= Entertainment, value="Entertainment", regex = True)
    cat = cat.replace(to_replace= Banking, value="Banking", regex = True)
    cat = cat.replace(to_replace= Culture, value="Culture", regex = True)


    cat = cat.replace(to_replace= Lodging, value="Lodging", regex = True)

    cat = cat.replace(to_replace= Infrastructure, value="Infrastructure", regex = True)
    cat = cat.replace(to_replace= Business, value="Business", regex = True)

    #Residential
    cat = cat.replace(regex=r'.*Food.*', value="Grocery")
    cat = cat.replace(regex=r'.*Grocery.*', value="Grocery")


    cat = cat.replace(regex=r'.*Auto.*', value="Home")
    cat = cat.replace(regex=r'.*School.*', value="Home")
    cat = cat.replace(regex=r'.*Residential.*', value="Home")
    cat = cat.replace(regex=r'.*Child.*', value="Home")
    cat = cat.replace(regex=r'.*Care.*', value="Home")
    cat = cat.replace(regex=r'.*Home.*', value="Home")
    cat = cat.replace(regex=r'.*Neighborhood.*', value="Home")

    cat = cat.replace(regex=r'.*Sports.*', value="Sports")
    cat = cat.replace(regex=r'.*Studio.*', value="Sports")
    cat = cat.replace(regex=r'.*Stables.*', value="Sports")
    cat = cat.replace(regex=r'.*Store.*', value="Sports")

    #Residential/Tourism
    cat = cat.replace(regex=r'.*Bar.*', value="Bar")
    cat = cat.replace(regex=r'.*Restaurant.*', value="Restaurant")

    cat = cat.replace(regex=r'.*Men.*', value="Shopping")
    cat = cat.replace(regex=r'.*Women.*', value="Shopping")

    cat = cat.replace(regex=r'.*Culture.*', value="Culture")
    cat = cat.replace(regex=r'.*Event.*', value="Culture")
    cat = cat.replace(regex=r'.*Concert.*', value="Culture")
    cat = cat.replace(regex=r'.*Art.*', value="Culture")

    cat = cat.replace(regex=r'.*Pool.*', value="Entertainment")
    cat = cat.replace(regex=r'.*Entertainment.*', value="Entertainment")


    #Tourism 
    cat = cat.replace(regex=r'.*Tour.*', value="Tourism")
    cat = cat.replace(regex=r'.*Hotel.*', value="Lodging")

    #Industrial 
    cat = cat.replace(regex=r'.*College.*', value="College")
    cat = cat.replace(regex=r'.*Business.*', value="Business")
    cat = cat.replace(regex=r'.*Bank.*', value="Banking")

    labels = labels.replace(regex=r'Grocery|Health|Home|Sports', value="Residential")
    labels = labels.replace(regex=r'Shopping|Bar|Restaurant|Culture|Banking|Entertainment|Transportation', value="RTBoth")
    labels = labels.replace(regex=r'Lodging|Tourism', value="Tourism")
    labels = labels.replace(regex=r'Business|Infrastructure|College|Factory', value="Industrial")

    return labels

## Get 

Run algorithm assigning weights for each neighorhood

In [ ]:
# each neighborhood has an overall residential, industrial and tousit count
# each neighborhood has an overall label 
# for each neighborhood 
#     count how many residential, industrial and tousit establishments there are. 
#     if an establishment is labeled as rtboth it gets a point in both residential and tourist. 
#     neighborhood is assigned label based on majority of establishments. 
#     if tie, break arbitrarly ---> priint these 

# cluster neighborhoods based on labels

In [30]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,16th Street Heights,Coffee Shop,Salvadoran Restaurant,Breakfast Spot,Bus Stop,Chinese Restaurant,Park,Bed & Breakfast,Greek Restaurant,Gym,Gymnastics Gym
1,0,Adams Morgan,Diner,Mediterranean Restaurant,Ethiopian Restaurant,Bar,Spa,Asian Restaurant,Ice Cream Shop,Coffee Shop,Cocktail Bar,Brazilian Restaurant
2,0,American University Park,Italian Restaurant,BBQ Joint,Yoga Studio,Filipino Restaurant,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field
3,0,Arboretum,Garden,Ice Cream Shop,Chinese Restaurant,Basketball Court,Fast Food Restaurant,Botanical Garden,Automotive Shop,Storage Facility,Gas Station,Nightclub
4,0,Barnaby Woods,Field,Gym / Fitness Center,Park,BBQ Joint,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant
5,0,Barry Farm,Bus Stop,Basketball Court,Intersection,Metro Station,Rental Car Location,Filipino Restaurant,Exhibit,Factory,Falafel Restaurant,Farmers Market
6,0,Bellevue,Grocery Store,Baseball Field,Playground,Pizza Place,Shoe Repair,Farmers Market,Ethiopian Restaurant,Event Service,Event Space,Exhibit
7,0,Benning,Discount Store,Liquor Store,Restaurant,Sandwich Place,Pharmacy,Donut Shop,Pizza Place,Fast Food Restaurant,Fried Chicken Joint,Food
8,2,Benning Ridge,Shipping Store,Insurance Office,Convenience Store,Burger Joint,Field,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant
9,0,Bloomingdale,Bus Stop,Yoga Studio,Grocery Store,Deli / Bodega,Convenience Store,Dive Bar,Pizza Place,Dog Run,Coffee Shop,Park


In [32]:
t= rename_labels(['Culture' 'Sports' 'Infrastructure' 'Grocery' 'Restaurant' 'Food' 'Home'
 'Banking' 'Shopping' 'Health' "Women's Store" 'Lodging' 'Bar' 'Pool'
 'Sports / Fitness Center' 'Flea Grocery' 'New Restaurant'
 'Latin Restaurant' 'Healthnish Restaurant' 'Transportation'
 'College Administrative Business' 'South Restaurant' 'Soccer Sports'
 'Neighborhood' 'Business' 'Concert Hall' 'Entertainment'
 'Child Care Service' 'Sculpture Culture' 'College Sports' "Men's Store"
 'Grocery Room' 'Christmas Grocery' 'Tour Provider'
 'General College & University' 'Barlic Art' 'Bankinging'
 'Residential Business (Apartment / Condo)' 'Sports Pool'
 'College Shopping' 'Stables' 'Food & Drink Shop' 'Sportsnastics Sports'
 'Government Business' 'Event Healthce' 'Lodging Bar' 'College Quad'
 'Entertainment Green' 'Pet Restaurant' 'Science Culture' 'Lodging Pool'
 'Photography Studio' 'Frame Store' 'Tourist Information Center'
 'Zoo Culture' 'Bubble Grocery Shop' 'College Stadium' 'College Library'
 'Event Service' 'Factory' 'Post Business'])
t

AttributeError: 'list' object has no attribute 'replace'

# KNN

Use KNN to cluster the neighborhoods into three groups by venue. 

In [23]:
# set number of clusters
kclusters = 3

dc_grouped_clustering = dc_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 2, 0], dtype=int32)

In [24]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [25]:
dc_merged = dc

# merge dc_grouped with dc_data to add latitude/longitude for each neighborhood
dc_merged = dc_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [26]:
address = 'Washington,DC'

geolocator = Nominatim(user_agent="dc_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 38.8949924, -77.0365581.


In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dc_merged['Latitude'], dc_merged['Longitude'], dc_merged['Neighborhood'], dc_merged['Cluster Labels']):
    if (math.isnan(cluster)):
        clus = 3
    else: 
        clus = int(cluster)
    label = folium.Popup('Neighborhood: '+ str(poi) + ' Cluster: ' + str(clus), parse_html=True)


    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[clus-1],
        fill=True,
        fill_color=rainbow[clus-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters